<a href="https://colab.research.google.com/github/javohiryoldashovBigData/IndianAirWaysPricePrediction_for2019/blob/main/IndianAirWaysPricePrediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import RandomizedSearchCV, cross_val_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor

In [35]:
train_data = pd.read_csv('./train_data.csv')
test_data = pd.read_csv('./test_data.csv')
submission_data = pd.read_csv('./sample_solution.csv')

 the data taken from this [link](https://www.kaggle.com/competitions/aviachipta-narxini-bashorat-qilish/data)

In [4]:
train_data.head()

,id,airline,flight,source_city,departure_time,stops,arrival_time,destination_city,class,duration,days_left,price
0,1,Vistara,UK-810,Bangalore,Early_Morning,one,Night,Mumbai,Economy,14.25,21,7212
1,2,SpiceJet,SG-5094,Hyderabad,Evening,zero,Night,Kolkata,Economy,1.75,7,5292
2,3,Vistara,UK-846,Bangalore,Morning,one,Evening,Delhi,Business,9.58,5,60553
3,4,Vistara,UK-706,Kolkata,Morning,one,Evening,Hyderabad,Economy,6.75,28,5760
4,5,Indigo,6E-5394,Chennai,Early_Morning,zero,Morning,Mumbai,Economy,2.00,4,10712


In [5]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   id                20000 non-null  int64  
 1   airline           20000 non-null  object 
 2   flight            20000 non-null  object 
 3   source_city       20000 non-null  object 
 4   departure_time    20000 non-null  object 
 5   stops             20000 non-null  object 
 6   arrival_time      20000 non-null  object 
 7   destination_city  20000 non-null  object 
 8   class             20000 non-null  object 
 9   duration          20000 non-null  float64
 10  days_left         20000 non-null  int64  
 11  price             20000 non-null  int64  
dtypes: float64(1), int64(3), object(8)
memory usage: 1.8+ MB


In [6]:
train_data.isnull().sum()

id                  0
airline             0
flight              0
source_city         0
departure_time      0
stops               0
arrival_time        0
destination_city    0
class               0
duration            0
days_left           0
price               0
dtype: int64

In [7]:
train_data.describe()

,id,duration,days_left,price
count,20000.000000,20000.000000,20000.000000,20000.000000
mean,10000.500000,12.177627,25.924150,20960.281700
std,5773.647028,7.157944,13.624874,22775.459535
min,1.000000,0.830000,1.000000,1105.000000
25%,5000.750000,6.830000,14.000000,4783.000000
50%,10000.500000,11.250000,26.000000,7425.000000
75%,15000.250000,16.080000,38.000000,42521.000000
max,20000.000000,38.580000,49.000000,114523.000000


In [8]:
X = train_data.drop('price', axis=1)
y = train_data['price']

In [9]:
num_att = ["duration", "days_left"]

cat_att =["airline", "flight", "source_city","departure_time","stops", "arrival_time", "destination_city", "class"]

num_pipeline = Pipeline([
 ("scaler", StandardScaler())
])

cat_pipeline = Pipeline ([
 ("ohe", OneHotEncoder(handle_unknown="ignore"))
])

full_pipeline = ColumnTransformer([
("num", num_pipeline, num_att),
("cat", cat_pipeline, cat_att)
])

X_pred = full_pipeline.fit_transform(X)
X_pred.toarray()

array([[ 0.28952803, -0.36141789,  0.        , ...,  1.        ,
         0.        ,  1.        ],
       [-1.45682723, -1.38897606,  0.        , ...,  0.        ,
         0.        ,  1.        ],
       [-0.36291029, -1.53577008,  0.        , ...,  0.        ,
         1.        ,  0.        ],
       ...,
       [ 3.23458153, -0.65500594,  0.        , ...,  0.        ,
         0.        ,  1.        ],
       [ 0.58012154, -0.36141789,  0.        , ...,  0.        ,
         0.        ,  1.        ],
       [ 0.84836171,  0.00556716,  0.        , ...,  0.        ,
         0.        ,  1.        ]])

# LinearRegression

In [10]:
LR_model = LinearRegression()
LR_model.fit(X_pred, y)

LinearRegression()

In [11]:
LR_prediction = LR_model.predict(X_pred)
print("Linear Regression:")
print("MAE", mean_absolute_error(LR_prediction,y))
print("RMSE", np.sqrt(mean_squared_error(LR_prediction,y)))
print("R-squared Score",r2_score(LR_prediction,y))

Linear Regression:
MAE 4145.0236685261325
RMSE 6060.580733564926
R-squared Score 0.9237898169395904


In [12]:
y = np.asarray(y).ravel()

# Random Forest Regressor

In [13]:
RF_model = RandomForestRegressor()
RF_model.fit(X_pred, y)

RandomForestRegressor()

In [14]:
RF_prediction = RF_model.predict(X_pred)
print("Random Foret Regressor:")
print("MAE", mean_absolute_error(RF_prediction,y))
print("RMSE", np.sqrt(mean_squared_error(RF_prediction, y)))
print("R-squared Score", r2_score(RF_prediction,y))

Random Foret Regressor:
MAE 565.8946360416667
RMSE 1232.3845639693625
R-squared Score 0.9970492225213776


# DecisionTreeRegressor

In [15]:
tree_model = DecisionTreeRegressor()
tree_model.fit(X_pred,y)

DecisionTreeRegressor()

In [16]:
DTree_prediction = tree_model.predict(X_pred)

print("Decision Tree Regressor:")
print("MAE", mean_absolute_error(DTree_prediction, y))
print("RMSE", np.sqrt(mean_squared_error(DTree_prediction,y)))
print("R-squared Score", r2_score(DTree_prediction,y))

Decision Tree Regressor:
MAE 0.8832
RMSE 42.13976684795491
R-squared Score 0.9999965764776922


# XGBoost Regressor

In [17]:
XGB_model = XGBRegressor()
XGB_model.fit(X_pred,y)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [18]:
XGB_prediction = XGB_model.predict(X_pred)
print("XGBoost Regressor:")
print("MAE", mean_absolute_error(XGB_prediction,y))
print("RMSE", np.sqrt(mean_squared_error(XGB_prediction,y)))
print("R-squared Score", r2_score(XGB_prediction,y))

XGBoost Regressor:
MAE 1880.3718255218505
RMSE 3199.0679785938523
R-squared Score 0.9795308414140089


# LGBM Regressor

In [19]:
LGBM_model = LGBMRegressor()
LGBM_model.fit(X_pred,y)

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001380 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 849
[LightGBM] [Info] Number of data points in the train set: 20000, number of used features: 274
[LightGBM] [Info] Start training from score 20960.281700


LGBMRegressor()

In [20]:
LGBM_prediction = LGBM_model.predict(X_pred)
print("LGBM Regressor:")
print("MAE", mean_absolute_error(LGBM_prediction,y))
print("RMSE", np.sqrt(mean_squared_error(LGBM_prediction,y)))
print("R-squared Score", r2_score(LGBM_prediction,y))

LGBM Regressor:
MAE 2072.342408590636
RMSE 3480.035201515424
R-squared Score 0.9758225335767411


# Cross Val Score

In [27]:
def display_scores(scores):
  print("Scores",scores)
  print("Mean", scores.mean())
  print("Std.dev", scores.std())

models = [LR_model,RF_model,tree_model,XGB_model,LGBM_model]
for model in models:
  print(f"{model} Regression")
  scores = cross_val_score(model, X_pred, y, scoring="neg_mean_squared_error", cv=5)
  rmse_scores = np.sqrt(-scores)
  print(display_scores(rmse_scores))

LinearRegression() Regression
Scores [6307.72362837 6210.16058457 6294.69349937 6397.00570944 6498.0641699 ]
Mean 6341.529518330864
Std.dev 98.15798749646798
None
RandomForestRegressor() Regression
Scores [3438.85416001 3409.13389584 3456.36865756 3546.38251286 3361.03353881]
Mean 3442.354553014314
Std.dev 61.232644431135725
None
DecisionTreeRegressor() Regression
Scores [4284.0865251  4469.57758231 4486.6562007  4554.56734248 4143.65921702]
Mean 4387.709373522797
Std.dev 151.4150809040342
None
XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_

Here we can see that the XGBoots Regressor model performs better than the others

# Submision


In [30]:
test_pred = full_pipeline.transform(test_data)

In [38]:
test_prediction = XGB_model.predict(test_pred)

In [39]:
submission_data['price'] = test_prediction
submission_data['price'] = submission_data['price'].apply(lambda x: "${:,.2f}".format(x))
submission_data.head()

,id,price
0,1,"$56,573.17"
1,2,"$59,782.50"
2,3,"$22,999.06"
3,4,"$3,086.57"
4,5,"$6,136.48"


In [40]:
submission_data.to_csv('My_sample_solution.csv', index=False)